In [2]:
import numpy as np
import cvxpy as cp

In [3]:
Q = np.array([
    [0, 0, 2,   0, 0, 9,   0, 5, 0],
    [0, 3, 0,   2, 0, 0,   1, 0, 0],
    [0, 0, 7,   0, 0, 5,   3, 0, 0],
    
    [7, 8, 0,   0, 0, 0,   0, 0, 0],
    [9, 0, 0,   0, 0, 0,   2, 6, 0],
    [0, 0, 0,   0, 9, 3,   0, 0, 5],
    
    [0, 0, 6,   0, 0, 0,   0, 9, 0],
    [0, 2, 1,   0, 0, 4,   5, 0, 3],
    [0, 0, 0,   7, 0, 0,   0, 0, 1],
])

In [4]:
for i in range(9):
    for j in range(9):
        if Q[i,j]==0:
            print('. ', end='')
        else:
            print(f'{int(Q[i,j].round())} ', end='')
        if j in [2, 5]:
            print('  ', end='')
    print('\n', end='')
    if i in [2, 5]:
        print('\n', end='')

. . 2   . . 9   . 5 . 
. 3 .   2 . .   1 . . 
. . 7   . . 5   3 . . 

7 8 .   . . .   . . . 
9 . .   . . .   2 6 . 
. . .   . 9 3   . . 5 

. . 6   . . .   . 9 . 
. 2 1   . . 4   5 . 3 
. . .   7 . .   . . 1 


In [5]:
def flattened_square_indices(i, j, size = 3, square_shape = 9):
    
    # Calculate the slice bounds for the square
    i_min, i_max = max(0, i + 1 - size // 2), min(square_shape, i+1 + size // 2 + 1)
    j_min, j_max = max(0, j + 1 - size // 2), min(square_shape, j+1 + size // 2 + 1)

    indices = []

    for i in range(i_min, i_max):
        for j in range(j_min, j_max):
            indices.append((square_shape*i + j))

    return indices

In [6]:
def sudoku_contraints(b):
    # Define the constraints for a sudoku board
    constraints = []

    # Each cell contains exactly one number
    for i in range(9):
        for j in range(9):
            constraints.append(cp.sum(b[i*9 + j, :]) == 1)

    # Each row contains each number exactly once
    for i in range(9):
        for k in range(9):
            horizontal_index = np.ravel_multi_index((i, np.arange(9)), (9,9))
            constraints.append(cp.sum(b[horizontal_index, k]) == 1)

    # Each column contains each number exactly once
    for j in range(9):
        for k in range(9):
            vertical_index = np.ravel_multi_index((np.arange(9), j), (9,9))
            constraints.append(cp.sum(b[vertical_index, k]) == 1)

    # Each 3x3 subgrid contains each number exactly once
    for i in range(3):
        for j in range(3):
            for k in range(9):
                square_index = flattened_square_indices(i*3, j*3, 3)
                constraints.append(cp.sum(b[square_index, k]) == 1)

    # Fix the known numbers
    for i in range(9):
        for j in range(9):
            if Q[i, j] != 0:
                constraints.append(b[i*9 + j, int(Q[i, j]-1)] == 1)
    
    return constraints

In [7]:
# Define the binary decision variables on a flattened sudoku board
b = cp.Variable((9*9, 9), boolean=True)

# Define the constraints
constraints = sudoku_contraints(b)

# Define the objective
obj = cp.Minimize(0)

# Define the problem
prob = cp.Problem(obj, constraints)

# Solve the problem
prob.solve()

print(f'Status: {prob.status}')

Status: optimal


In [ ]:
# Print the solution

for i in range(9):
    for j in range(9):
        for k in range(9):
            if b[i*9 + j, k].value == 1:
                    print(f'{k+1} ', end='')

        if j in [2, 5]:
            print('  ', end='')
    print('\n', end='')
    if i in [2, 5]:
        print('\n', end='')

6 4 2   3 1 9   8 5 7 
5 3 8   2 7 6   1 4 9 
1 9 7   4 8 5   3 2 6 

7 8 5   6 2 1   9 3 4 
9 1 3   5 4 7   2 6 8 
2 6 4   8 9 3   7 1 5 

3 7 6   1 5 8   4 9 2 
8 2 1   9 6 4   5 7 3 
4 5 9   7 3 2   6 8 1 
